In [1]:
%pip install -qU pypdf langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.0 MB/s eta 0:00:00


In [5]:
%pip install langchain sentence-transformers chromadb together faiss-cpu streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.2 MB/s eta 0:00:00


In [6]:
%%writefile app.py
import os
import tempfile
import streamlit as st
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from together import Together

# Define the Together API key
TOGETHER_API_KEY = os.environ.get('TOGETHER_API_KEY')
client = Together(api_key=TOGETHER_API_KEY)

# Function to load and process documents from a list of uploaded files
def load_documents_from_files(files):
    documents = []
    for file in files:
        # Save the uploaded file to a temporary directory
        with tempfile.NamedTemporaryFile(delete=False) as temp_file:
            temp_file.write(file.read())  # Write the content of the uploaded file
            temp_file_path = temp_file.name  # Get the path of the saved file
        # Use the saved file path with PyPDFLoader
        loader = PyPDFLoader(temp_file_path)
        docs = loader.load()
        documents.extend(docs)
        # Clean up the temporary file after processing
        os.remove(temp_file_path)
    return documents

# Function to process the documents and create the FAISS vector store
def create_vector_store_from_files(files):
    # Load and process documents from the uploaded files
    documents = load_documents_from_files(files)

    # Split the document into smaller chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(documents)

    # Use Sentence Transformers for embeddings
    embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

    # Create a FAISS vector store using the SentenceTransformer embeddings
    vectorstore = FAISS.from_documents(documents=splits, embedding=embedding_model)

    return vectorstore

# Create a retriever function to get relevant documents based on the query
def retrieve_information(query, files):
    # Create the vector store only once when the files are uploaded
    vectorstore = create_vector_store_from_files(files)

    # Retrieve information based on the query
    retriever = vectorstore.as_retriever()
    retrieved_info = retriever.invoke(query)

    # Return the first relevant document's content or a message if no results
    if retrieved_info:
        return retrieved_info[0].page_content
    else:
        return "No relevant information found."

# Function to send the retrieved context and query to the LLM and get the answer
def query_llm(query, retrieved_text):
    prompt = f"""
    The user is asking the following question: "{query}"
    Below is some relevant information from the documents that could help in answering the question:
    {retrieved_text}

    Now, based on the information above, please provide a short and human-like answer to the user query. Limit your response to a few lines, summarizing the key points mostly from the given context.
    """

    try:
        response = client.chat.completions.create(
            model="meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=100,  # Control the response length
            temperature=0.7,
            top_p=0.9,
            top_k=50,
            repetition_penalty=1.0,
            stop=["<|eot_id|>", "<|eom_id|>"],
            stream=True
        )

        full_response = ""
        for token in response:
            if hasattr(token, 'choices'):
                full_response += token.choices[0].delta.content

        return full_response

    except Exception as e:
        return f"Error: {str(e)}"

# Streamlit App
st.title("Customer Support AI")
st.markdown("Upload your company's knowledge base and get instant 24/7 customer support")

# File uploader for PDFs
uploaded_files = st.file_uploader("Choose PDF files", type="pdf", accept_multiple_files=True)

if uploaded_files:
    # Display User's query
    query = st.text_input("Ask a question:")

    if query:
        # Show the user's question
        st.markdown(f"**User:** {query}")

        # Call the function to retrieve information based on the query
        retrieved_info = retrieve_information(query, uploaded_files)

        # Call the LLM with the retrieved information and the user's query
        answer = query_llm(query, retrieved_info)

        # Display Chatbot's response
        st.markdown(f"**Chatbot:** {answer}")



Overwriting app.py


In [4]:
!wget -q -O - ipv4.icanhazip.com

34.48.138.235


In [8]:
! streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸⠼⠴
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.48.138.235:8501

⠦Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹your url is: https://yellow-sheep-cover.loca.lt
  Stopping...
^C


In [9]:
#upload your custom knowledge base and replace the query!
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from together import Together

TOGETHER_API_KEY = "YOUR-API-KEY"

# Load your document
file_path = "/content/KnowledgeBase.pdf"
loader = PyPDFLoader(file_path)
docs = loader.load()

# Split the document into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Use Sentence Transformers for embeddings
embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")  # You can choose a different model here

# Create a FAISS vector store using the SentenceTransformer embeddings
vectorstore = FAISS.from_documents(documents=splits, embedding=embedding_model)

# Create a retriever from the vector store
retriever = vectorstore.as_retriever()

# Define the query
query = "What is visualization?"

# Retrieve relevant information based on the query
retrieved_info = retriever.get_relevant_documents(query)

# Combine the retrieved text into a single string
retrieved_text = "\n".join([info.page_content for info in retrieved_info])

# Initialize the Together API client
client = Together(api_key=TOGETHER_API_KEY)

# Define the prompt to pass the query and the retrieved context to the LLM
prompt = f"""
The user is asking the following question: "{query}"
Below is some relevant information from the documents that could help in answering the question:
{retrieved_text}

Now, based on the information above, please provide a short and human-like answer to the user query. Limit your response to a few lines, summarizing the key points mostly from the given context.
"""

# Send the request to the LLM
response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo",
    messages=[{"role": "user", "content": prompt}],
    max_tokens=100,  # Control the response length
    temperature=0.7,
    top_p=0.9,
    top_k=50,
    repetition_penalty=1.0,
    stop=["<|eot_id|>", "<|eom_id|>"],
    stream=True  # Enable streaming for live response
)

# Collect and print the response from the LLM
full_response = ""
for token in response:
    if hasattr(token, 'choices'):
        full_response += token.choices[0].delta.content

# Display the final answer
print("Answer:", full_response)


<ipython-input-9-28b3b7f2e985>:22: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")  # You can choose a different model here
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-9-28b3b7f2e985>:34: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_info = retriever.get_relevant_documents(query)


Answer: Visualization is the process of representing data in a visual format to understand and analyze it. It involves several stages, including acquiring and parsing data, filtering out irrelevant information, applying statistical methods, choosing a visual representation, refining it for clarity, and adding interaction for user exploration. The goal of visualization is to make data more readable, understandable, and engaging, often using graphic design methods and statistical techniques to discern patterns and trends.
